# Aplicando o ferramental de tempo de passagem


In [25]:
import pandas as pd
import numpy as np
from scipy.stats import gamma

In [58]:
floresta = pd.read_csv('https://raw.githubusercontent.com/Gorgens/treelab/master/manejo/InventarioMultinivelArea2.csv')
floresta['comum'] = floresta['comum'].str.capitalize()
floresta = floresta[floresta['comum'] != 'Morta']
floresta = floresta.drop(['parcela', 'arv', 'UTM_Easting', 'UTM_Northing', 'Hcom.2013', 'Htot.2013'], axis=1)
floresta.rename(columns={'dap.2013': 'dap1'}, inplace=True)
floresta.sample(10)

,comum,cientifico,familia,dap1
633,Sucupira preta,Bowdichia nitida,Fabaceae,40.2
656,Burra leiteira,Sapium marmieri,Euphorbiaceae,12.0
198,Inga,Inga marginata,Fabaceae,10.8
356,Mapatirana,Pourouma minor,Urticaceae,42.2
95,Matamata,Eschweilera parviflora,Lecythidaceae,42.1
75,Envira preta,Bocageopsis pleiosperma,Annonaceae,15.8
431,Abiurana,Pouteria gongrijpii,Sapotaceae,33.0
465,Embauba,Cecropia ficifolia,Urticaceae,12.0
317,A?i,Euterpe oleracea,Arecaceae,11.7
443,Matamata,Eschweilera parviflora,Lecythidaceae,45.6


## Centro de classe

In [60]:
# intervaloClasse = 10.0
# diametroMin = 10.0
# classeMax = ((max(area2vivas['dap.2013']) // intervaloClasse) + 1) * intervaloClasse

# intervalos = list(np.arange(diametroMin, classeMax, intervaloClasse))
# # intervalos = list(map((lambda x: x - 0.001), intervalos))
# intervalos = [(x - 0.001) for x in intervalos]
# centros = [round(x + intervaloClasse/2.0, 0) for x in intervalos[:-1]]

# intervaloMedicao = 2

# area2vivas = area2vivas[area2vivas['dap.2013'] >= diametroMin]
# area2vivas['centroClasse'] = pd.cut(area2vivas['dap.2013'], bins=intervalos, labels=centros)
# area2vivas.sample(10)

In [52]:
comunidade = floresta.shape[0]
ingresso = sum(np.random.binomial(1, 0.02, comunidade))
ingressantes = pd.DataFrame()
ingressantes['cientifico'] = area2vivas['cientifico'].sample(ingresso)
ingressantes['dap.2013'] = 10
new = pd.concat([area2vivas, ingressantes])

,parcela,arv,comum,cientifico,familia,dap.2013,Hcom.2013,Htot.2013,UTM_Easting,UTM_Northing,centroClasse
373,NaN,NaN,NaN,Pourouma minor,NaN,10.0,NaN,NaN,NaN,NaN,NaN
578,NaN,NaN,NaN,Cecropia ficifolia,NaN,10.0,NaN,NaN,NaN,NaN,NaN
347,NaN,NaN,NaN,Cordia scabrifolia,NaN,10.0,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,Tetragastris altissima,NaN,10.0,NaN,NaN,NaN,NaN,NaN
469,NaN,NaN,NaN,Cecropia ficifolia,NaN,10.0,NaN,NaN,NaN,NaN,NaN


In [84]:
tempo = 5
floresta['dapFuturo'] = floresta['dap1']
floresta['dTempo'] = 0

for t in np.arange(1, tempo+1, 1):
  print('tempo', t-1, ':', floresta.shape[0])
  # Morrer
  comunidade = floresta.shape[0]
  morrer = np.random.binomial(1, 0.04, comunidade)

  # Crescer
  crescer = np.random.binomial(1, 0.8132, comunidade)
  crescer2 = []
  for x, i in zip(crescer, morrer):
    crescer2.append(x if i != 1 else 0)

  # Incremento 

  incremento = np.random.gamma( 0.48, 1, comunidade)
  incremento2 = crescer2*incremento

  # simulacao = pd.DataFrame({'morrer': morrer,
  #               'crescer': crescer,
  #               'crescer2': crescer2,
  #             'incremento': incremento,
  #             'incremento2': incremento2})
  floresta['morta'] = morrer
  floresta['inc'] = incremento2
  floresta['dTempo'] = t
  floresta['dapFuturo'] = floresta['dapFuturo'] + floresta['inc']
  floresta = floresta[floresta['morta'] != 1]
  print('pós mortalidade: ', floresta.shape[0])

  comunidade = floresta.shape[0]
  ingresso = sum(np.random.binomial(1, 0.04, comunidade))
  ingressantes = pd.DataFrame()
  ingressantes['cientifico'] = floresta['cientifico'].sample(ingresso)
  ingressantes['dap1'] = 10
  floresta = pd.concat([floresta, ingressantes])
  print('tempo', t, ':', floresta.shape[0])

tempo 0 : 451
pós mortalidade:  438
tempo 1 : 453
tempo 1 : 453
pós mortalidade:  429
tempo 2 : 448
tempo 2 : 448
pós mortalidade:  432
tempo 3 : 443
tempo 3 : 443
pós mortalidade:  426
tempo 4 : 437
tempo 4 : 437
pós mortalidade:  415
tempo 5 : 434


In [74]:
floresta.sample(10)

,comum,cientifico,familia,dap1,dapFuturo,morta,inc,dTempo
479,Abiu rosadinho,Pouteria anibifolia,Sapotaceae,17.0,17.929197,0.0,0.929197,1.0
417,Embauba,Cecropia ficifolia,Urticaceae,17.7,17.714822,0.0,0.014822,1.0
84,Matamata,Eschweilera parviflora,Lecythidaceae,52.3,52.300033,0.0,0.000033,1.0
458,Embauba,Cecropia ficifolia,Urticaceae,13.0,13.282613,0.0,0.282613,1.0
567,Jatereu,Lecythis idatimon,Lecythidaceae,24.4,24.400000,0.0,0.000000,1.0
486,Embauba,Cecropia ficifolia,Urticaceae,12.5,12.515889,0.0,0.015889,1.0
268,Taxi preto,Tachigali myrmecophila,Fabaceae,16.7,16.933457,0.0,0.233457,1.0
176,Pente de macaco,Apeiba echinata,Malvaceae,13.9,13.964691,0.0,0.064691,1.0
405,Barrote,Tetragastris altissima,Burseraceae,36.5,36.500000,0.0,0.000000,1.0
310,Ni,NI,NI,16.4,16.927049,0.0,0.527049,1.0
